# ETL

In [6]:
#Importamos los paquete necesarios:
import os
import pandas as pd
import glob ##New package learned
import shutil ##New package learned   
from sqlalchemy import create_engine

# Extract:

In [7]:
def from_xlsx(ventas_file):
    ''' Extrae los datos de un documento XLSX y devuele un DataFrame 
    IN: XLSX 
    OUT: DataFrame
    '''

    return pd.read_excel(ventas_file)

In [8]:
def from_plano(ventas_file):
    ''' Extrae los datos de un documento TXT, con separador "|" y devuele un DataFrame 
    IN: TXT 
    OUT: DataFrame
    '''

    return pd.read_csv(ventas_file, sep='|')

In [9]:
def from_json(ventas_file):
    ''' Extrae los datos de un documento JSON y devuele un DataFrame 
    IN: JSON 
    OUT: DataFrame
    '''

    return pd.read_json(ventas_file)

In [10]:
def from_csv(ventas_file):
    ''' Extrae los datos de un documento CSV y devuele un DataFrame 
    IN: CSV 
    OUT: DataFrame
    '''

    return pd.read_csv(ventas_file)

In [ ]:
def extract():
    '''
    Extraer datos de diferentes fuentes y generar un DataFrame conjunto:

    IN: TXT, JSON, XLSX y CSV
    OUT: DataFrame
    '''
    #Definimos un DataFrame que contendrá todos los datos:
    total_sales = pd.DataFrame(columns=["id_venta", "fecha", "producto", "precio", "cantidad"])

    #Identificar el formato de datos:
    for file in os.listdir('dirty_data/'):
        
        if file[-5:] == ".xlsx":
            xlsx_df = from_xlsx("dirty_data/"+file)
        
        elif file[-4:] == ".txt":
            txt_df = from_plano(os.path.abspath(file))
        
        elif file[-5:] == ".json":
            json_df = from_json(os.path.abspath(file))

        elif file[-4:] == ".csv":
            csv_df = from_csv(os.path.abspath(file))
            
            total_sales = pd.concat([total_sales, csv_df], join='inner')
        


        

        

        #Dirigirlo a la extraccion correcta:
    
    #Sumar los datos al DataFrame total:
    #print(total_sales.head())
    #Devolver el total en un DataFrame:
    return total_sales

# Transform:

# Load:

# Ejecución:

In [14]:
extract()

NameError: name 'csv_df' is not defined